<h1>al_bench</h1>
Example use of the al_bench Active Learning Benchmark Tool

In [ ]:
# Install needed packages
!pip install h5py numpy tensorflow
!pip install -e /tf/notebooks/al_bench

<h2>Dataset</h2>
Fetch a dataset of 4598 feature vectors of length 1280 and their 4598 labels.

In [1]:
import al_bench as alb
import h5py as h5
import numpy as np

filename = "../test/TCGA-A2-A0D0-DX1_xmin68482_ymin39071_MPP-0.2500.h5py"
with h5.File(filename) as ds:
    my_features = np.array(ds["features"])
    print(
        f"Read in {my_features.shape[0]} feature vectors of length {my_features.shape[1]}."
    )
    my_labels = np.array(ds["labels"])
    print(f"Read in {my_labels.shape[0]} labels for the feature vectors.")
my_label_definitions = [
    {
        0: {"description": "other"},
        1: {"description": "tumor"},
        2: {"description": "stroma"},
        3: {"description": "infiltrate"},
    }
]
my_dataset_handler = alb.dataset.GenericDatasetHandler()
my_dataset_handler.set_all_features(my_features)
my_dataset_handler.set_all_label_definitions(my_label_definitions)
my_dataset_handler.set_all_labels(my_labels)

Read in 4598 feature vectors of length 1280.
Read in 4598 labels for the feature vectors.


<h2>Model</h2>
Build a model that we will train.  We choose a TensorFlow model, but we could have chosen a PyTorch model.

In [2]:
import tensorflow as tf

number_of_categories = len(my_label_definitions[0])
number_of_features = my_features.shape[1]
hidden_units = 128
dropout = 0.3
my_model = tf.keras.models.Sequential(
    [
        tf.keras.Input(shape=(number_of_features,)),
        tf.keras.layers.Dense(hidden_units, activation="relu"),
        tf.keras.layers.Dropout(dropout, noise_shape=None, seed=20220909),
        tf.keras.layers.Dense(number_of_categories, activation="softmax"),
    ],
    name=(
        f"{number_of_categories}_labels_from_{number_of_features}_features_with_"
        f"dropout_{dropout}"
    ),
)
my_model_handler = alb.model.TensorFlowModelHandler()
my_model_handler.set_model(my_model)

2022-09-09 09:42:27.302380: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-09 09:42:29.578625: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1922] Ignoring visible gpu device (device: 1, name: Quadro P400, pci bus id: 0000:a6:00.0, compute capability: 6.1) with core count: 2. The minimum required count is 8. You can adjust this requirement with the env var TF_MIN_GPU_MULTIPROCESSOR_COUNT.
2022-09-09 09:42:29.579257: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-09 09:

<h2>Active Learning Strategy</h2>
Choose an active learning strategy to test

In [3]:
# my_strategy_handler = alb.strategy.RandomStrategyHandler()
my_strategy_handler = alb.strategy.LeastConfidenceStrategyHandler()
# my_strategy_handler = alb.strategy.LeastMarginStrategyHandler()
# my_strategy_handler = alb.strategy.EntropyStrategyHandler()

my_strategy_handler.set_dataset_handler(my_dataset_handler)
my_strategy_handler.set_model_handler(my_model_handler)
my_strategy_handler.set_learning_parameters(
    label_of_interest=0,  # We've supplied only one label per feature vector
    maximum_iterations=5,
    number_to_select_per_iteration=20,
)

<h2>Run the benchmarking tool</h2>

In [4]:
# Assume that we start with nothing labeled
currently_labeled_examples = set()
my_strategy_handler.run(currently_labeled_examples)

Predicting for 4598 examples
108/144 [=====================>........] - ETA: 0s

2022-09-09 09:42:33.114182: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


144/144 [==============================] - 2s 1ms/step
Training with 20 examples
Epoch 1/10
1/1 [==============================] - 1s 626ms/step - loss: 1.4931 - accuracy: 0.2000
Epoch 2/10
1/1 [==============================] - 0s 6ms/step - loss: 0.8506 - accuracy: 0.7500
Epoch 3/10
1/1 [==============================] - 0s 6ms/step - loss: 0.5248 - accuracy: 0.9500
Epoch 4/10
1/1 [==============================] - 0s 6ms/step - loss: 0.3505 - accuracy: 0.9500
Epoch 5/10
1/1 [==============================] - 0s 6ms/step - loss: 0.2492 - accuracy: 0.9500
Epoch 6/10
1/1 [==============================] - 0s 6ms/step - loss: 0.1993 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 6ms/step - loss: 0.1808 - accuracy: 0.9500
Epoch 8/10
1/1 [==============================] - 0s 5ms/step - loss: 0.1177 - accuracy: 0.9500
Epoch 9/10
1/1 [==============================] - 0s 6ms/step - loss: 0.0780 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s